# Reproduce DDGemb method

In [1]:
import pandas as pd
import numpy as np


In [2]:
path = 'data/'
df_fireprot = pd.read_csv(path + 'fireprotdb_results.csv')


C:\Users\badea\AppData\Local\Temp\ipykernel_14028\2789855006.py:2: DtypeWarning: Columns (23,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fireprot = pd.read_csv(path + 'fireprotdb_results.csv')


In [4]:
df_fireprot.head()

,experiment_id,protein_name,uniprot_id,pdb_id,chain,position,wild_type,mutation,ddG,dTm,...,technique,technique_details,pH,tm,notes,publication_doi,publication_pubmed,hsw_job_id,datasets,sequence
0,LL000001,Haloalkane dehalogenase,P59336,1CQW,A,245,V,L,NaN,2.1,...,NaN,NaN,NaN,52.5,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
1,LL000002,Haloalkane dehalogenase,P59336,1CQW,A,95,L,V,NaN,-0.4,...,NaN,NaN,NaN,50.0,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
2,LL000004,Haloalkane dehalogenase,P59336,1CQW,A,176,C,F,NaN,5.2,...,NaN,NaN,NaN,55.6,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
3,LL000005,Haloalkane dehalogenase,P59336,1CQW,A,171,G,Q,NaN,3.1,...,NaN,NaN,NaN,53.5,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
4,LL000006,Haloalkane dehalogenase,P59336,1CQW,A,148,T,L,NaN,1.1,...,NaN,NaN,NaN,51.5,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...


In [5]:
print(df_fireprot.columns)

Index(['experiment_id', 'protein_name', 'uniprot_id', 'pdb_id', 'chain',
       'position', 'wild_type', 'mutation', 'ddG', 'dTm', 'is_curated', 'type',
       'derived_type', 'interpro_families', 'conservation', 'is_essential',
       'correlated_positions', 'is_back_to_consensus', 'secondary_structure',
       'asa', 'is_in_catalytic_pocket', 'is_in_tunnel_bottleneck', 'b_factor',
       'method', 'method_details', 'technique', 'technique_details', 'pH',
       'tm', 'notes', 'publication_doi', 'publication_pubmed', 'hsw_job_id',
       'datasets', 'sequence'],
      dtype='object')


In [7]:
protein_counts_1 = df_fireprot["protein_name"].value_counts().reset_index()
protein_counts_1

,protein_name,count
0,Subtilisin-chymotrypsin inhibitor-2A,12276
1,Tyrosine-protein kinase Fyn,2488
2,Halohydrin dehalogenase,2480
3,ADHA,2404
4,Immunoglobulin G-binding protein G,2220
...,...,...
205,"Ferredoxin, heterocyst",1
206,Non-specific lipid-transfer protein,1
207,S-adenosylmethionine synthase isoform type-1,1
208,Tetracycline repressor protein class D,1


In [14]:
columns_to_keep = ['experiment_id', 'protein_name', 'uniprot_id', 'pdb_id', 'chain',
       'position', 'wild_type', 'mutation', 'ddG', 'sequence']
df_fireprot = df_fireprot[-df_fireprot['ddG'].isna()]
df_subset = df_fireprot[columns_to_keep]

In [18]:
df_subset[df_subset["protein_name"].isna()].pdb_id.unique()

array(['2IMM', '1YYX'], dtype=object)

In [16]:
print("The dataset contains {} rows".format(len(df_subset)))

The dataset contains 39177 rows


In [20]:
from transformers import AutoTokenizer, EsmModel
import torch

In [21]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

d:\Harvard\Spring\BMI707\BMI707FinalProject\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\badea\.cache\huggingface\hub\models--facebook--esm2_t6_8M_UR50D. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [22]:
model = EsmModel.from_pretrained("facebook/esm2_t6_8M_UR50D")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
seq_test = df_subset.iloc[0]["sequence"]

inputs = tokenizer(seq_test, return_tensors="pt", padding=True, truncation=True, max_length=1024)
with torch.no_grad():
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    pooled_output = last_hidden_states[:,1:-1,:].squeeze(0)
    print(pooled_output.shape)

torch.Size([448, 320])


In [35]:
pooled_output = pooled_output.numpy()

In [36]:
pooled_output.shape, len(seq_test)

((448, 320), 448)

In [41]:
import torch.nn as nn

class DDGPredictor(nn.Module):
    def __init__(self, 
                esm_model_name="facebook/esm2_t6_8M_UR50D", 
                embedding_dim=320, 
                conv_channels=128, 
                heads=4, 
                ffn_dim=256):
        super(DDGPredictor, self).__init__()
        
        # Load pretrained ESM2 model
        self.esm_model = EsmModel.from_pretrained(esm_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(esm_model_name)
        
        # Freeze ESM model if needed
        for param in self.esm_model.parameters():
            param.requires_grad = False
        
        # 1D Conv Layer
        self.conv1d = nn.Conv1d(
            in_channels=embedding_dim, 
            out_channels=conv_channels, 
            kernel_size=3, 
            padding=1
            )
        
        # Multi-Head Attention
        self.attention = nn.MultiheadAttention(
            embed_dim=conv_channels, 
            num_heads=heads, 
            batch_first=True)
        
        # Position-wise FFN
        self.ffn = nn.Sequential(
            nn.Linear(conv_channels, ffn_dim),
            nn.ReLU(),
            nn.Linear(ffn_dim, conv_channels)
        )
        
        # Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        
        # Dense layers for final regression
        self.regressor = nn.Sequential(
            nn.Linear(conv_channels * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def get_embeddings(self, seqs):
        # Tokenize
        tokens = self.tokenizer(seqs, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.esm_model(**tokens)
        # Extract per-residue embeddings
        return outputs.last_hidden_state[:,1:-1,:]  # shape: (batch, L, 320)
    
    def forward(self, wt_seqs, mut_seqs):
        # Get ESM embeddings
        emb_wt = self.get_embeddings(wt_seqs)
        emb_mut = self.get_embeddings(mut_seqs)
        
        # Take difference
        d = emb_wt - emb_mut  # shape: (B, L, 320)
        
        # Conv1D expects (B, C, L)
        c = self.conv1d(d.transpose(1, 2)).transpose(1, 2)  # shape: (B, L, conv_channels)
        
        # Multihead attention: q, k, v = c
        m, _ = self.attention(c, c, c)
        
        # Residual connection
        z = c + m
        
        # Feedforward with residual
        p = self.ffn(z)
        f = z + p
        
        # Pooling
        f_t = f.transpose(1, 2)  # (B, C, L)
        gp = self.global_avg_pool(f_t).squeeze(-1)  # (B, C)
        gm = self.global_max_pool(f_t).squeeze(-1)  # (B, C)
        
        conc = torch.cat([gp, gm], dim=1)  # (B, 2C)
        
        # Final regression
        ddg_pred = self.regressor(conc).squeeze(-1)
        return ddg_pred

In [42]:
model = DDGPredictor()

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#define wt_seqs_batch and mut_seqs_batch, ddG_true

In [ ]:
model = DDGPredictor()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    pred = model(wt_seqs_batch, mut_seqs_batch)
    loss = loss_fn(pred, ddG_true)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()